In [28]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd

In [29]:
#========
# Setup
#========
df = pd.DataFrame({
    "date": ["2025-12-01" , "2025-12-02" , "2025-12-04" , "bad_date" , "2025-12-07"] ,
    "sales": [120 , 125 , 110 , 130 , 140] ,
    "returns": [4 , 5 , 3 , 6 , 2] ,
})
df

,date,sales,returns
0,2025-12-01,120,4
1,2025-12-02,125,5
2,2025-12-04,110,3
3,bad_date,130,6
4,2025-12-07,140,2


In [30]:
#================================
# Case 1) Parse dates correctly
#================================
df["date"] = pd.to_datetime(df["date"] , errors = "coerce")

df_valid = df.dropna(subset = ["date"]).copy()
df_valid

,date,sales,returns
0,2025-12-01,120,4
1,2025-12-02,125,5
2,2025-12-04,110,3
4,2025-12-07,140,2


In [31]:
#==========================================
# Case 2) Extract date features with .dt
#==========================================
df_valid["year"] = df_valid["date"].dt.year
df_valid["month"] = df_valid["date"].dt.month
df_valid["day_name"] = df_valid["date"].dt.day_name()
df_valid["quarter"] = df_valid["date"].dt.quarter
df_valid["is_month_end"] = df_valid["date"].dt.is_month_end
ts = df_valid.sort_values("date").set_index("date")
ts

,sales,returns,year,month,day_name,quarter,is_month_end
date,,,,,,,
2025-12-01,120,4,2025,12,Monday,4,False
2025-12-02,125,5,2025,12,Tuesday,4,False
2025-12-04,110,3,2025,12,Thursday,4,False
2025-12-07,140,2,2025,12,Sunday,4,False


In [32]:
#=====================================
# Case 3) Build a complete calendar
#=====================================
full_idx = pd.date_range(ts.index.min() , ts.index.max() , freq = "D")
ts_full = ts.reindex(full_idx)
ts_full

,sales,returns,year,month,day_name,quarter,is_month_end
2025-12-01,120.0,4.0,2025.0,12.0,Monday,4.0,False
2025-12-02,125.0,5.0,2025.0,12.0,Tuesday,4.0,False
2025-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-04,110.0,3.0,2025.0,12.0,Thursday,4.0,False
2025-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-07,140.0,2.0,2025.0,12.0,Sunday,4.0,False


In [33]:
#==============================
# Case 4) Time-based slicing
#==============================
dec = ts_full.loc["2025-12"]
dec

window = ts_full.loc["2025-12-02":"2025-12-06"]
window

,sales,returns,year,month,day_name,quarter,is_month_end
2025-12-01,120.0,4.0,2025.0,12.0,Monday,4.0,False
2025-12-02,125.0,5.0,2025.0,12.0,Tuesday,4.0,False
2025-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-04,110.0,3.0,2025.0,12.0,Thursday,4.0,False
2025-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-07,140.0,2.0,2025.0,12.0,Sunday,4.0,False


,sales,returns,year,month,day_name,quarter,is_month_end
2025-12-02,125.0,5.0,2025.0,12.0,Tuesday,4.0,False
2025-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-04,110.0,3.0,2025.0,12.0,Thursday,4.0,False
2025-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
#==================================
# Case 5) Time-aware comparisons
#==================================
ts_full["sales_lag1"] = ts_full["sales"].shift(1)
ts_full["sales_change"] = ts_full["sales"].diff()
ts_full[["sales" , "sales_lag1" , "sales_change"]].round(2)

,sales,sales_lag1,sales_change
2025-12-01,120.0,NaN,NaN
2025-12-02,125.0,120.0,5.0
2025-12-03,NaN,125.0,NaN
2025-12-04,110.0,NaN,NaN
2025-12-05,NaN,110.0,NaN
2025-12-06,NaN,NaN,NaN
2025-12-07,140.0,NaN,NaN


In [35]:
#============================================
# Case 6) Time zones (localize vs convert)
#============================================
events = pd.DataFrame({
    "event_time": pd.to_datetime([
        "2025-12-01 09:15:00" , "2025-12-01 13:40:00" , "2025-12-01 18:05:00" ,
    ]) ,
    "event": ["open" , "checkout" , "close"] ,
})

events["utc_time"] = events["event_time"].dt.tz_localize("UTC")
events["local_time"] = events["utc_time"].dt.tz_convert("America/New_York")
events[["event" , "event_time" , "utc_time" , "local_time"]]

,event,event_time,utc_time,local_time
0,open,2025-12-01 09:15:00,2025-12-01 09:15:00+00:00,2025-12-01 04:15:00-05:00
1,checkout,2025-12-01 13:40:00,2025-12-01 13:40:00+00:00,2025-12-01 08:40:00-05:00
2,close,2025-12-01 18:05:00,2025-12-01 18:05:00+00:00,2025-12-01 13:05:00-05:00
